<a href="https://colab.research.google.com/github/pmckim1/Capstone/blob/main/_ACT_UN_peacekeeping_scrape_12_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- scrape PDFS


In [ ]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
# import scraping libraries
import requests
from bs4 import BeautifulSoup
from google.colab import files


In [ ]:
#url to download pdfs
#url="https://peacekeeping.un.org/en/troop-and-police-contributors"

#04_mission_and_country_54_september_2022

In [ ]:
import os
from google.colab import drive
import sys
import pandas as pd
import json
import PyPDF2
# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder

abspath_curr = '/content/drive/MyDrive/'

MessageError: ignored

In [ ]:
# Requests URL and get response object
import pandas as pd
# creating a pdf file object
pdfFileObj = open(abspath_curr + '04_mission_and_country_47_feb_2022.pdf', 'rb')
# creating a pdf reader object
pdfReader = PyPDF2.PdfReader(pdfFileObj)
# printing number of pages in pdf file
# print(pdfReader.numPages)

document_text=""
# creating a page object
for idx in range(len(pdfReader.pages)):
  pageObj = pdfReader.pages[idx]
  print(pageObj)
  # extracting text from page
  document_text += pageObj.extract_text()

# closing the pdf file object
pdfFileObj.close()



In [ ]:
clean_document_text = [line.strip() for line in document_text.splitlines() if line]
for line in clean_document_text:
  print(line)

In [ ]:
clean_document_text = [line.strip() for line in document_text.splitlines() if line]

mission_types = ["BINUH", "MINURSO", "MINUSCA", "MINUSMA", "MONUSCO", "UNAMA","UNAMI", "UNDOF", "UNFICYP", "UNIFIL", "UNISFA",
                 "UNITAMS", "UNMHA", "UNMIK", "UNMISS", "UNMOGIP", "UNOWAS", "UNSMIL",
                 "UNSOM", "UNSOS", "UNTSO", "UNVMC"]
personel_types = ["Individual Police", "Experts on Mission", "Staff Officer"]

df = pd.DataFrame(columns = ["Mission","Country", "Personnel Type", "Male", "Female", "Total"])


line_iter = iter(clean_document_text)

next_row = {
  "Mission": None,
  "Country": None,
  "Personnel Type": None,
  "Male": None,
  "Female": None,
  "Total": None,
}

def is_number(line):
  try:
    int(line)
    return True
  except:
    return False

mission = None
country = None
idx = 0
while idx < len(clean_document_text) - 1:
    idx = idx + 1
    line = clean_document_text[idx]
    # print(line)
    if line in mission_types:
      mission = line
      next_row["Mission"] = mission
      #print(f"Its a mission line: {mission}")
      continue
    elif not mission:
      #print(f"Not in a mission: {line}")
      continue

    if line.startswith("Total "):
      mission = None
      continue

    if is_number(clean_document_text[idx+1]):
      next_row['Personnel Type'] =  clean_document_text[idx]
      next_row["Male"] = clean_document_text[idx+1]
      next_row["Female"] = clean_document_text[idx+2]
      next_row["Total"] = clean_document_text[idx+3]
      idx += 3
      df = df.append(next_row, ignore_index=True)
    else:
      country = line
      next_row["Country"] = country

#print(df)

df = df[df['Personnel Type'] != 'Page']


df["Mission_Name"]=df["Mission"].replace({'MINURSO': 'Western Sahara',
                                     "MINUSCA": 'CAR',
                                     'MINUSMA': 'Mali',
                                    'MONUSCO': 'DRC',
                                    'UNAMA': 'Afghanistan',
                                     'UNAMI': 'Iraq',
                                     'UNDOF': 'Golan Heights',
                                    'UNFICYP': 'Cyprus',
                                     'UNIFIL': 'Lebanon',
                                     'UNISFA': 'Abyei',
                                     'UNITAMS': 'Sudan',
                                     'UNMHA': 'Yemeni',
                                    'UNMIK': 'Kosovo',
                                     'UNMISS': 'South Sudan',
                                     'UNMOGIP': 'Kashmir',
                                     'UNOWAS': 'Dakar',
                                     'UNSMIL': 'Libya',
                                     'UNSOM': 'Somalia',
                                     'UNSOS': 'Somalia',
                                      'UNTSO': 'Jerusalem',
                                     'UNVMC': 'Colombia'})

In [ ]:
df.Mission.unique()

In [ ]:
df.head()

In [ ]:
df.to_csv("peace_keeping_feb.csv")

In [ ]:
from google.colab import files
files.download("peace_keeping_feb.csv")